In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "cpi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "30"

    if len(sheet_names) == 11:
        return "Sheet2"

    raise RuntimeError(sheet_names)

In [3]:
en_to_vn = {
    "CONSUMER PRICE INDEXES": "CHỈ SỐ GIÁ TIÊU DÙNG",
    "Food and catering services": "Hàng ăn và dịch vụ ăn uống",
    "Grain food": "Lương thực",
    "Foodstuff": "Thực phẩm",
    "Outdoor eating and drinking": "Ăn uống ngoài gia đình",
    "Drinks and tobacco": "Đồ uống và thuốc lá",
    "Textile, footgear and hats": "May mặc, mũ nón và giày dép ",
    "Housing and construction materials": "Nhà ở và vật liệu xây dựng",
    "Family appliances and tools": "Thiết bị và đồ dùng gia đình",
    "Medicaments and health service": "Thuốc và dịch vụ y tế",
    "Medical service": "Dịch vụ y tế",
    "Transport": "Giao thông",
    "Postal and communicational service": "Bưu chính viễn thông",
    "Education": "Giáo dục",
    "Educational service": "Dịch vụ giáo dục",
    "Culture, entertainment and tourism": "Văn hoá, giải trí và du lịch",
    "Others": "Hàng hóa và dịch vụ khác",
    "GOLD PRICE INDEXES": "CHỈ SỐ GIÁ VÀNG",
    "US DOLLAR PRICE INDEXES": "CHỈ SỐ GIÁ ĐÔ LA MỸ",
    "CORE INFLATION": "LẠM PHÁT CƠ BẢN",
}

In [4]:
def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Hàng ăn và dịch vụ ăn uống")
                | (df[df.columns[1]] == "Food and catering services")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Lương thực")
                | (df[df.columns[1]] == "Grain food")
                | (df[df.columns[1]] == " Lương thực")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if len(df.columns) == 7:
        df = df.drop(columns=[df.columns[1], df.columns[3], df.columns[5]])
    elif len(df.columns) == 6 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[4]])
    elif len(df.columns) == 6:
        df = df.drop(columns=[df.columns[1], df.columns[3]])
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1]])
        df["A"] = df[df.columns[1]]
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1]])
    else:
        raise RuntimeError(df.columns)

    df.columns = ["Hàng hóa", "YOY", "MOM", "YTD-YOY"]

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "CHỈ SỐ GIÁ TIÊU DÙNG")
                    | ((df[df.columns[0]] == "CONSUMER PRICE INDEXES"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "CONSUMER PRICE INDEXES"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    # df = df[pd.to_numeric(df["MOM"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YOY"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]

    df["MOM"] = df["MOM"].astype(float) - 100
    df["YOY"] = df["YOY"].astype(float) - 100
    df["YTD-YOY"] = df["YTD-YOY"].astype(float) - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("../datas/2019-12-27-03-Bieu-2019.xlsx")
get_cpi(xl, find_sheet_name(xl.sheet_names), "2019-12-27")

,Hàng hóa,YOY,MOM,YTD-YOY,Date
8,CHỈ SỐ GIÁ TIÊU DÙNG,5.234549,1.3990,2.791355,2019-12-27
9,Hàng ăn và dịch vụ ăn uống,9.166051,3.4162,4.058663,2019-12-27
10,Lương thực,0.815858,0.4471,0.455607,2019-12-27
11,Thực phẩm,12.494944,4.4099,5.075439,2019-12-27
12,Ăn uống ngoài gia đình,5.331673,2.4369,3.341627,2019-12-27
13,Đồ uống và thuốc lá,2.270155,0.2454,1.991732,2019-12-27
14,"May mặc, mũ nón và giày dép",1.488504,0.3330,1.735623,2019-12-27
15,Nhà ở và vật liệu xây dựng,5.115138,0.4268,3.034216,2019-12-27
16,Thiết bị và đồ dùng gia đình,1.339185,0.1387,1.353589,2019-12-27
17,Thuốc và dịch vụ y tế,2.987244,0.0346,3.835458,2019-12-27


In [5]:
import os
import contextlib

dfs_cpi_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_cpi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_cpi_origin.append(df_cpi)
    except Exception as e:
        print(file_name)
        raise e

In [6]:
dfs_cpi = pd.concat(dfs_cpi_origin, ignore_index=True, axis=0)
dfs_cpi["Date"] = pd.to_datetime(dfs_cpi["Date"], dayfirst=False)
dfs_cpi.head(n=20)

,Hàng hóa,YOY,MOM,YTD-YOY,Date
0,CHỈ SỐ GIÁ TIÊU DÙNG,2.633365,0.285000,3.880696,2024-09-29
1,Hàng ăn và dịch vụ ăn uống,3.942224,0.924000,3.996824,2024-09-29
2,Lương thực,8.373684,0.771200,14.227573,2024-09-29
3,Thực phẩm,3.239588,1.060900,2.312815,2024-09-29
4,Ăn uống ngoài gia đình,3.835410,0.651500,4.033576,2024-09-29
5,Đồ uống và thuốc lá,2.299965,0.025200,2.447279,2024-09-29
6,"May mặc, mũ nón và giày dép",1.167287,0.151100,1.480736,2024-09-29
7,Nhà ở và vật liệu xây dựng,4.369913,0.519500,5.334865,2024-09-29
8,Thiết bị và đồ dùng gia đình,1.165822,0.144200,1.223911,2024-09-29
9,Thuốc và dịch vụ y tế,8.280087,0.047500,7.455521,2024-09-29


In [7]:
convert_hh = {
    "CHỈ SỐ GIÁ TIÊU DÙNG": "CHỈ SỐ GIÁ TIÊU DÙNG",
    "Hàng ăn và dịch vụ ăn uống": "Hàng ăn và dịch vụ ăn uống",
    "Lương thực": "Lương thực",
    "Thực phẩm": "Thực phẩm",
    "Ăn uống ngoài gia đình": "Ăn uống ngoài gia đình",
    "Đồ uống và thuốc lá": "Đồ uống và thuốc lá",
    "May mặc, mũ nón và giày dép": "May mặc, mũ nón và giày dép",
    "Nhà ở và vật liệu xây dựng": "Nhà ở và vật liệu xây dựng",
    "Thiết bị và đồ dùng gia đình": "Thiết bị và đồ dùng gia đình",
    "Thuốc và dịch vụ y tế": "Thuốc và dịch vụ y tế",
    "Dịch vụ y tế": "Dịch vụ y tế",
    "Giao thông": "Giao thông",
    "Bưu chính viễn thông": "Bưu chính viễn thông",
    "Giáo dục": "Giáo dục",
    "Dịch vụ giáo dục": "Dịch vụ giáo dục",
    "Văn hoá, giải trí và du lịch": "Văn hoá, giải trí và du lịch",
    "Hàng hóa và dịch vụ khác": "Hàng hóa và dịch vụ khác",
    "CHỈ SỐ GIÁ VÀNG": "CHỈ SỐ GIÁ VÀNG",
    "CHỈ SỐ GIÁ ĐÔ LA MỸ": "CHỈ SỐ GIÁ ĐÔ LA MỸ",
    "LẠM PHÁT CƠ BẢN": "LẠM PHÁT CƠ BẢN",
    "May mặc, giày dép và mũ nón": "May mặc, mũ nón và giày dép",
    "Đồ dùng và dịch vụ khác": "Hàng hóa và dịch vụ khác",
    "May mặc, mũ nón và giày dép ": "May mặc, mũ nón và giày dép",
    "LẠM PHÁT CƠ BẢN(*)": "LẠM PHÁT CƠ BẢN",
}

for hh in dfs_cpi["Hàng hóa"].unique().tolist():
    assert hh in convert_hh.keys(), hh

dfs_cpi["Hàng hóa"] = dfs_cpi["Hàng hóa"].map(convert_hh)
dfs_cpi

,Hàng hóa,YOY,MOM,YTD-YOY,Date
0,CHỈ SỐ GIÁ TIÊU DÙNG,2.633365,0.2850,3.880696,2024-09-29
1,Hàng ăn và dịch vụ ăn uống,3.942224,0.9240,3.996824,2024-09-29
2,Lương thực,8.373684,0.7712,14.227573,2024-09-29
3,Thực phẩm,3.239588,1.0609,2.312815,2024-09-29
4,Ăn uống ngoài gia đình,3.835410,0.6515,4.033576,2024-09-29
...,...,...,...,...,...
2560,Dịch vụ giáo dục,12.450000,0.0000,12.450000,2014-01-29
2561,"Văn hoá, giải trí và du lịch",2.900000,0.2100,2.900000,2014-01-29
2562,Hàng hóa và dịch vụ khác,4.910000,0.6300,4.910000,2014-01-29
2563,CHỈ SỐ GIÁ VÀNG,-24.430000,-1.8200,-24.430000,2014-01-29


In [8]:
dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YOY"] + 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "MOM"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "MOM"] + 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YTD-YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"].str.contains("LẠM PHÁT CƠ BẢN"), "YTD-YOY"] + 100
)

In [9]:
dfs_cpi.to_csv("../csv/cpi.csv")

In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "CHỈ SỐ GIÁ TIÊU DÙNG")
        | (dfs_cpi["Hàng hóa"] == "LẠM PHÁT CƠ BẢN")
    )
    & (dfs_cpi["Date"] > "2015-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "Lương thực")
        | (dfs_cpi["Hàng hóa"] == "Nhà ở và vật liệu xây dựng")
        | (dfs_cpi["Hàng hóa"] == "Giao thông")
        | (dfs_cpi["Hàng hóa"] == "Giáo dục")
        | (dfs_cpi["Hàng hóa"] == "Thuốc và dịch vụ y tế")
    )
    & (dfs_cpi["Date"] > "2021-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()